<a href="https://colab.research.google.com/github/AzDevops143/Ai-assignment/blob/main/AIfirst_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import heapq
import itertools

class ManuscriptPuzzle:
    def __init__(self, start_state, goal_state):
        self.start_state = tuple(start_state)
        self.goal_state = tuple(goal_state)
        # Pre-calculating goal positions for O(1) heuristic lookup
        self.goal_map = {val: (i // 3, i % 3) for i, val in enumerate(self.goal_state)}

    def get_h(self, state):
        """Calculates Manhattan Distance (Heuristic)."""
        distance = 0
        for i, val in enumerate(state):
            if val != 'B':
                curr_r, curr_c = i // 3, i % 3
                goal_r, goal_c = self.goal_map[val]
                distance += abs(curr_r - goal_r) + abs(curr_c - goal_c)
        return distance

    def solve(self):
        # Tie-breaker to fix the TypeError in your image
        counter = itertools.count()
        start_b = self.start_state.index('B')

        # open_set: (f, tie_breaker, g, state, b_idx, path_data)
        # path_data stores (state, g, h, f) for visualization
        h_start = self.get_h(self.start_state)
        open_set = [(h_start, next(counter), 0, self.start_state, start_b, [])]
        visited = {self.start_state: 0}

        while open_set:
            f, _, g, current, b_idx, path = heapq.heappop(open_set)

            # Record current state info for output
            current_path = path + [(current, g, f-g, f)]

            if current == self.goal_state:
                return current_path

            r, c = b_idx // 3, b_idx % 3
            for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]: # Up, Down, Left, Right
                nr, nc = r + dr, c + dc
                if 0 <= nr < 3 and 0 <= nc < 3:
                    target_idx = nr * 3 + nc
                    new_state = list(current)
                    new_state[b_idx], new_state[target_idx] = new_state[target_idx], new_state[b_idx]
                    neighbor = tuple(new_state)

                    # Cost of movement is 1 unit of System Energy per move
                    new_g = g + 1
                    if neighbor not in visited or new_g < visited[neighbor]:
                        visited[neighbor] = new_g
                        h = self.get_h(neighbor)
                        heapq.heappush(open_set, (new_g + h, next(counter), new_g, neighbor, target_idx, current_path))
        return None

def display_stages(path):
    """Prints each stage with g(n), h(n), and f(n) details."""
    if not path:
        print("No solution found.")
        return

    for i, (state, g, h, f) in enumerate(path):
        header = "INITIAL STATE" if i == 0 else f"STAGE {i}"
        print(f"--- {header} ---")
        print(f"  g(n) = {g} (Path Cost)")
        print(f"  h(n) = {h} (Heuristic)")
        print(f"  f(n) = {f} (Total Cost)")

        # Display as 3x3 grid
        for row in range(0, 9, 3):
            print(f"    {state[row]} {state[row+1]} {state[row+2]}")
        print("-" * 20)

# --- EXECUTION ---
# Goal state from your requirements: 1 2 3 / 4 5 6 / 7 8 B
goal_layout = [1, 2, 3, 4, 5, 6, 7, 8, 'B']

# Example Scrambled State
initial_layout = [1, 2, 3,'B', 4, 5, 7, 8, 6]

solver = ManuscriptPuzzle(initial_layout, goal_layout)
full_path = solver.solve()
display_stages(full_path)

--- INITIAL STATE ---
  g(n) = 0 (Path Cost)
  h(n) = 3 (Heuristic)
  f(n) = 3 (Total Cost)
    1 2 3
    B 4 5
    7 8 6
--------------------
--- STAGE 1 ---
  g(n) = 1 (Path Cost)
  h(n) = 2 (Heuristic)
  f(n) = 3 (Total Cost)
    1 2 3
    4 B 5
    7 8 6
--------------------
--- STAGE 2 ---
  g(n) = 2 (Path Cost)
  h(n) = 1 (Heuristic)
  f(n) = 3 (Total Cost)
    1 2 3
    4 5 B
    7 8 6
--------------------
--- STAGE 3 ---
  g(n) = 3 (Path Cost)
  h(n) = 0 (Heuristic)
  f(n) = 3 (Total Cost)
    1 2 3
    4 5 6
    7 8 B
--------------------
